### TESTE  ONE-WAY  ANOVA + TESTE POST HOC DE TUKEY PARA AMOSTRAS INDEPENDENTES

Comparar a média de **dois ou mais** grupos ou amostras independentes. Amostras independentes (não pareadas) são selecionadas aleatoriamente de modo que os valores de observações não dependam de valores de outras observações.

#### PREMISSAS:

- Premissa 1: Amostras independentes
- Premissa 2: Distribuição Normal
- Premissa 3: Variâncias iguais      
- Premissa 4: A variável dependente (Y) é contínua e a explanatória (X) é categorica.


#### RESULTADO DA ANÁLISE DE VARIÂNCIA (ANOVA):

- As colunas F-value e Pr(>F) correspondentes ao p-value do teste.
- Se o p-value é maior que o nível de significância, pode-se concluir que não existem diferenças significativas entre os grupos destacados no sumário do modelo.


#### COMPRAÇÃO MÚLTIPLA DE PARES ENTRE AS MÉDIAS DOS GRUPOS:

No teste ANOVA unilateral, um valor p significativo indica que algumas das médias dos grupos são diferentes, mas não se sabe quais pares de grupos são diferentes. É possível realizar múltiplas comparações de pares para determinar se a diferença
média entre pares específicos do grupo é estatisticamente significativa.

Comparações de pares múltiplos de Tukey, um teste post-hoc, pode ser usado para mais de 3 grupos. Se o teste ANOVA é significativo, podemos calcular Tukey HSD (Tukey Honest Significant Differences, função R: TukeyHSD()) para realizar múltiplas comparações de pares entre as médias dos grupos. A função TukeyHD() usa a ANOVA ajustada como argumento.

#### RESULTADO TUKEYHDS:

- diff: diferença entre as médias dos dois grupos
- lwr, upr: o ponto inferior e superior do intervalo de confiança em 95% (padrão)
- p adj: valor p após o ajuste para as comparações múltiplas.


In [1]:
# DESATIVA NOTAÇÃO CIENTÍFICA
options(scipen = 999)

In [2]:
# DESATIVAR MENSAGENS DE WARNINGS (warn = -1) E ATIVAR (warn = 0)
options(warn = -1)

In [3]:
# CARREGANDO PACOTES
suppressPackageStartupMessages({
library(dplyr)
library(multcomp)
})

#### CARREGANDO DATASET

Dataset PlantGrowth do R com os esultados de um experimento para comparar os rendimentos (medidos pelo peso seco das plantas) obtidos sob um controle e duas condições de tratamento diferentes.

#### TESTE:
Existe alguma diferença significativa entre os pesos médios das plantas nas 3 condições experimentais?

In [4]:
dados_plantas <- PlantGrowth
colnames(dados_plantas) <- c("Peso", "Grupo")
glimpse(dados_plantas)
cat("\nGrupo de plantas:", levels(dados_plantas$Grupo))

Rows: 30
Columns: 2
$ Peso  <dbl> 4.17, 5.58, 5.18, 6.11, 4.50, 4.61, 5.17, 4.53, 5.33, 5.14, 4.81~
$ Grupo <fct> ctrl, ctrl, ctrl, ctrl, ctrl, ctrl, ctrl, ctrl, ctrl, ctrl, trt1~

Grupo de plantas: ctrl trt1 trt2

In [5]:
dados_sumarizados <- group_by(dados_plantas, Grupo) %>%
  summarise(n = n(),
            Media = mean(Peso, na.rm = TRUE),
            DP = sd(Peso, na.rm = TRUE))
dados_sumarizados

Grupo,n,Media,DP
ctrl,10,5.032,0.5830914
trt1,10,4.661,0.7936757
trt2,10,5.526,0.4425733


#### TESTE ANOVA UNILATERAL

In [6]:
analise_variancia <- aov(Peso ~ Grupo, data = dados_plantas)
summary(analise_variancia)

            Df Sum Sq Mean Sq F value Pr(>F)  
Grupo        2  3.766  1.8832   4.846 0.0159 *
Residuals   27 10.492  0.3886                 
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

#### TESTE POST HOC DE TUKEY

In [7]:
TukeyHSD(analise_variancia)

  Tukey multiple comparisons of means
    95% family-wise confidence level

Fit: aov(formula = Peso ~ Grupo, data = dados_plantas)

$Grupo
            diff        lwr       upr     p adj
trt1-ctrl -0.371 -1.0622161 0.3202161 0.3908711
trt2-ctrl  0.494 -0.1972161 1.1852161 0.1979960
trt2-trt1  0.865  0.1737839 1.5562161 0.0120064


#### TESTE POST HOC DE TUKEY - FUNÇÃO glht()

In [8]:
summary(glht(analise_variancia, linfct = mcp(Grupo = "Tukey")))


	 Simultaneous Tests for General Linear Hypotheses

Multiple Comparisons of Means: Tukey Contrasts


Fit: aov(formula = Peso ~ Grupo, data = dados_plantas)

Linear Hypotheses:
                 Estimate Std. Error t value Pr(>|t|)  
trt1 - ctrl == 0  -0.3710     0.2788  -1.331   0.3909  
trt2 - ctrl == 0   0.4940     0.2788   1.772   0.1980  
trt2 - trt1 == 0   0.8650     0.2788   3.103   0.0119 *
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
(Adjusted p values reported -- single-step method)


Existe um diferença significativa entre trt2 e trt1 com um Pr(>|t|) = 0.0121